# Sluzba CtiOSGdal

Nejdrive naimportujeme knihovny:

In [1]:
import os
import sys
import sqlite3
import tempfile
from shutil import copyfile

# Moduly jsou v jinem adresari, je tedy nutne tento adresar pridat do sys.path
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
   sys.path.append(module_path)

from services.ctiOS.gdal import CtiOSGdal

Pote nadefinujeme parametry nutne pro inicializaci knihovny:

In [2]:
# pristupove testovaci udaje k sluzbam WSDP
username = "WSTEST"
password = "WSHESLO"
# rozbaleny VFK soubor ve forme sqlite databaze
db_path = os.path.join('../', module_path, 'data', 'input', 'ctios_template.db')
print(db_path)

/home/linwe/pywsdp/data/input/ctios_template.db


### Zpracovani cele tabulky OPSUB

Nejprve nas bude zajimat zpracovani cele tabulky OPSUB - tedy doplneni udaju ke vsem radkum v teto tabulce. V databazi jsou tyto udaje zatim nevyplnene:

In [3]:
con = sqlite3.connect(db_path)
cur = con.cursor()

for row in cur.execute("SELECT * FROM OPSUB"):
    print(row)
    
con.close()

('64R47PeqD9SONivn8apY/lJWp7PMqXzY9utJl+M+BfUUirKmMZliVuxYKEtLpIva6glOyFHdvIofu9Hwu2HKl1m5M6qSyaHfCuQu6GdIy0E=', None, None, None, None, None, None, None, None, None, 'OPO', 3, 1, 1, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 1)
('cWAf13LtjvC9SIY05qIFaek9K/RpuqJao0AMXiHsDYFjM2eS251ouFycwesABoDs2wWu+kFTmYqxhJyeR7e+WrdGQBbrYHk4M21Ai3xMSF4=', None, None, None, None, None, None, None, None, None, 'OFO', 2, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 2)
('psGq9l14Woi29AYN5bHwdxU5WmX1vyRzVSJuucNo5e2D8193gu/to3VObZbd3vc0L+fDzX+X5nL42Wu8iePZgJhokFKOM8uNqezVZiQQ6P0=', None, None, None, None, None, None, None, None, None, 'OFO', 2, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 3)
('r/e02ptfcJ47n0

Abychom neztratili puvodni nevyplnena data, udelame si bokem kopii databaze, kterou pak budeme editovat:

In [4]:
temp_db = tempfile.mkstemp()[1]
copyfile(db_path, temp_db)
print(temp_db)

/tmp/tmp324uikoa


Zavolame knihovnu a ziskame vsechny identifikatory z databaze ve forme seznamu:


In [5]:
ctios = CtiOSGdal(username=username,
                  password=password,
                  db_path=temp_db)
posidents = ctios.get_parameters_from_db()
print(posidents)
len(posidents)

['<v2:pOSIdent>WpAGVIbtrw3p6Ic2ebTbXNCwsz0FKsnGB9V9QeaExKo21vI1yF5xcnK/ftMPvAe0P43ooFP1+vRsJWe0rET59I1aQW/uq+Ca3bHca+9xTcc=</v2:pOSIdent>', '<v2:pOSIdent>xocTMRuuiHIYwNN3v/RUpArt2Fhetx6zLzs1TUM4gw+ObLyU2HJkAPQ5WdENo82O2DPHBJrt2XiurT7d3j3H5mEqs6IWf89iXlFyWJLkDow=</v2:pOSIdent>', '<v2:pOSIdent>l0raZybOozQ95JWgTnaQpiplmmtdzAp2RiExSTwc+CmrRcDkW8jPHrWwYA33OP9DV3647Ea9f04G8Wyca2N6FJccnQQNFaY66A5TWLwA8dc=</v2:pOSIdent>', '<v2:pOSIdent>5wQRil9Nd5KIrn5KWTf8+ucfKEsayz4LR/ZrAucExc8z+4hWyMyh+ABIewfIBvgQMxxF8/2HqFMn0OE/IphQisnADW27BjQ8Cn2Ui2i+xLM=</v2:pOSIdent>', '<v2:pOSIdent>00ep7V5fFBZoFMeOfcAOPMRtu0G9svLcsDyC/fYkJk3EbFIqbw2kpRaC1jjS1V8bjacEwrp/SdxXZXdtwDzFj3vKzXvHF7k7HiDpVtv6fPAUIS/cKLcyZ7viQd7pE5FE</v2:pOSIdent>', '<v2:pOSIdent>8bg+CXEGHarWp2oK+IfpUamKDfCVz85r0uBnUCjzPdNJ6AK2a4ybpIROxdjinogDSMGKQB9jeCyt5BLEOx9qsj3iksX7gk0kyMb4XwEPBO4=</v2:pOSIdent>', '<v2:pOSIdent>UKcYWvUUTpNi8flxUzlm+TSnnprgWLVUHQLYF9LrVQpOOqQ6wycE7XBa1xVt+SzU+NR+i4CvbqbaTivPkjaaQbFOZw8LEOjjFYowWGpfJHKUl6UeeOCVx+jaGjtgvaWo</v

108

Nad seznamem zavolame metodu .process, ktera k zadanym identifikatorum aktualizuje v tabulce osobni udaje:

In [6]:
ctios.process(posidents)

ctiOS        - CRITICAL - WSDP REQUEST ERROR - 404 Client Error: Not Found for url: https://wsdptrial.cuzk.cz/trial/ws/ctios/2.8/ctios


WSDPRequestError: (<WSDPLogger ctiOS (DEBUG)>, HTTPError('404 Client Error: Not Found for url: https://wsdptrial.cuzk.cz/trial/ws/ctios/2.8/ctios',))

Pri posilani pozadavku na sluzbu mohou byt na strane WSDP sluzby odhaleny chyby ve vstupnim souboru: neplatny identifikator, expirovany identifikator a opravneny subjekt neexistuje. Jak muzeme videt, na konci zpracovani se nam vypise jednoducha statistika zpracovavanych subjektu obsahujici i statistiku chyb. Tuto statistiku je treba po kazdem zpracovani zkontrolovat.

 Nyni uz se muzeme podivat do naplnene databaze:

Podivame se do naplnene databaze:

In [ ]:
con = sqlite3.connect(temp_db)
cur = con.cursor()

for row in cur.execute("SELECT * FROM OPSUB"):
    print(row)
    
con.close()

Skvele! Databazi se podarilo aktualizovat. :-)

### Vyber pseudonymizovanych identifikatoru

Pokud nepotrebujeme zpracovavat celou tabulku, muzeme si vybrat jen nektere pseudonymizovane identifikatory.

Zase si udelame kopii databaze:

In [ ]:
temp_db = tempfile.mkstemp()[1]
copyfile(db_path, temp_db)
print(temp_db)

Zavolame si knihovnu a pri ziskavani identifikatoru z databaze pouzijeme parametr "sql". Zajima nas pouze 10 prvnich identifikatoru:

In [ ]:
ctios = CtiOSGdal(username=username,
                  password=password,
                  db_path=temp_db)
sql = "SELECT * FROM OPSUB order by ID LIMIT 10"
posidents = ctios.get_parameters_from_db(sql) # pouzijeme parametr sql
print(posidents)

Nad seznamem zavolame metodu .process, ktera k zadanym identifikatorum aktualizuje v tabulce osobni udaje:

In [ ]:
ctios.process(posidents)

Podivame se na aktualizovanou databazi:

In [ ]:
con = sqlite3.connect(temp_db)
cur = con.cursor()

for row in cur.execute("SELECT * FROM OPSUB order by ID LIMIT 10"):
    print(row)
    
con.close()

Je dobre vedet, ze identifikatory nemusime ziskavat z databaze, ale muzeme je take ziskat z testoveho souboru, kde je oddelovacem carka:

In [ ]:
txt_path = os.path.join(module_path, 'services', 'ctiOS', 'data', 'input', 'ctios_all.txt')
posidents = ctios.get_parameters_from_txt(txt_path)
print(posidents)

Nebo je muzeme rovnou inicializovat jako seznam, ktery predame jako argument do funkce ctios.process().